<a href="https://colab.research.google.com/github/j1-aggie/big-data-/blob/main/Level_one_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:15 http://ppa.la

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-01-12 22:53:23--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.30MB/s    in 0.7s    

2021-01-12 22:53:25 (1.30 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://etl-s3-colab.s3.us-east-2.amazonaws.com/employee.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("employee.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

+---+-----------+--------+--------------------+--------+--------------+------+---------+--------+--------------+----------------+--------------------+------------+
|_c0|Employee ID|   Email|            Position|     DOB|Marital Status|Gender|Hire Date|Salaried|Vacation Hours|Sick Leave Hours|  Encrypted Password|    Modified|
+---+-----------+--------+--------------------+--------+--------------+------+---------+--------+--------------+----------------+--------------------+------------+
|  1|  295847284|    ken0|Chief Executive O...| 1/29/69|             S|     M|  1/14/09|       1|            99|              69|F01251E5-96A3-448...|6/30/14 0:00|
|  2|  245797967|  terri0|Vice President of...|  8/1/71|             S|     F|  1/31/08|       1|             1|              20|45E8F437-670D-440...|6/30/14 0:00|
|  3|  509647174|roberto0| Engineering Manager|11/12/74|             M|     M| 11/11/07|       1|             2|              21|9BBBFB2C-EFBB-421...|6/30/14 0:00|
|  4|  112457891

## Drop duplicates and incomplete rows

In [ ]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

292
291
289


## Examine the schema

In [ ]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Employee ID: integer (nullable = true)
 |-- Email: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Hire Date: string (nullable = true)
 |-- Salaried: integer (nullable = true)
 |-- Vacation Hours: integer (nullable = true)
 |-- Sick Leave Hours: integer (nullable = true)
 |-- Encrypted Password: string (nullable = true)
 |-- Modified: string (nullable = true)



## Rename columns

In [ ]:
df1 = df.withColumnRenamed("Employee ID", "employee_id") \
        .withColumnRenamed("Email", "email") \
        .withColumnRenamed("Gender", "gender") \
        .withColumnRenamed("Hire Date", "hire_date") \
        .withColumnRenamed("DOB", "dob") \
        .withColumnRenamed("Encrypted Password", "password")
df1.show(5)

+---+-----------+-------+--------------------+--------+--------------+------+---------+--------+--------------+----------------+--------------------+------------+
|_c0|employee_id|  email|            Position|     dob|Marital Status|gender|hire_date|Salaried|Vacation Hours|Sick Leave Hours|            password|    Modified|
+---+-----------+-------+--------------------+--------+--------------+------+---------+--------+--------------+----------------+--------------------+------------+
| 23|  767955365|  mary0|Marketing Specialist| 9/13/62|             M|     F|  1/12/09|       0|            46|              43|90899830-127E-487...|6/30/14 0:00|
|142|  163347032|olinda0|Production Techni...|  4/4/70|             S|     F|   3/4/10|       0|             3|              21|6456CEC3-9B1E-4EB...|6/30/14 0:00|
|171|  746373306| david3|Production Techni...|  8/3/81|             S|     M|   1/2/09|       0|            32|              36|84C358D4-2493-452...|6/30/14 0:00|
|253|  407505660| lind

## Create a new DataFrame for employee info

In [ ]:
employee_personal_info = df1.select(["employee_id", "email", "gender", "hire_date", "dob"])
employee_personal_info.show(5)

+-----------+-------+------+---------+--------+
|employee_id|  email|gender|hire_date|     dob|
+-----------+-------+------+---------+--------+
|  767955365|  mary0|     F|  1/12/09| 9/13/62|
|  163347032|olinda0|     F|   3/4/10|  4/4/70|
|  746373306| david3|     M|   1/2/09|  8/3/81|
|  407505660| linda2|     F| 12/17/09|11/30/70|
|  658797903|  gigi0|     F|  1/16/09| 1/21/79|
+-----------+-------+------+---------+--------+
only showing top 5 rows



## Write DataFrame to RDS

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://bootcamp-1.cjr5lzjymj5a.us-east-2.rds.amazonaws.com:5432/student_activity"
config = {"user":"postgres",
          "password": "postgres",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table

employee_personal_info.write.jdbc(url=jdbc_url, table='employee_personal_info', mode=mode, properties=config)

## Create a new DataFrame for employee passwords

In [ ]:
employee_password = df1.select(["employee_id", "password"])
employee_password.show(5)

+-----------+--------------------+
|employee_id|            password|
+-----------+--------------------+
|  767955365|90899830-127E-487...|
|  163347032|6456CEC3-9B1E-4EB...|
|  746373306|84C358D4-2493-452...|
|  407505660|ED06F9F4-2123-4C4...|
|  658797903|50B6CDC6-7570-47E...|
+-----------+--------------------+
only showing top 5 rows



## Write DataFrame to RDS

In [ ]:
# Write DataFrame to table

employee_password.write.jdbc(url=jdbc_url, table='employee_password', mode=mode, properties=config)